## Task 2.1: EMA Indicators


In [1]:
import pandas as pd

spot_df = pd.read_csv("../data/clean/spot_clean.csv")
spot_df["timestamp"] = pd.to_datetime(spot_df["timestamp"])
spot_df = spot_df.sort_values("timestamp")


In [3]:
spot_df["ema_5"] = spot_df["close"].ewm(span=5, adjust=False).mean()
spot_df["ema_15"] = spot_df["close"].ewm(span=15, adjust=False).mean()

spot_df.head()


,timestamp,open,high,low,close,volume,ema_5,ema_15
99,2025-08-22,226.17,229.09,225.410,227.76,42477811,227.760000,227.760000
98,2025-08-25,226.48,229.30,226.230,227.16,30983133,227.560000,227.685000
97,2025-08-26,226.87,229.49,224.690,229.31,54575107,228.143333,227.888125
96,2025-08-27,228.61,230.90,228.260,230.49,31259513,228.925556,228.213359
95,2025-08-28,230.82,233.41,229.335,232.56,38074700,230.137037,228.756689


EMA(5) represents short-term momentum  
EMA(15) represents medium-term trend  
EMA crossover will be used for signal generation


## Task 2.2: Options Greeks and Implied Volatility


In [4]:
import pandas as pd
import numpy as np

options_df = pd.read_csv("../data/clean/options_clean.csv")
atm_df = pd.read_csv("../data/clean/atm_strikes.csv")

options_df.head()


,CONTRACT_D,PREVIOUS_S,OPEN_PRICE,HIGH_PRICE,LOW_PRICE,CLOSE_PRIC,SETTLEMENT,NET_CHANGE,OI_NO_CON,TRADED_QUA,TRD_NO_CON,UNDRLNG_ST,NOTIONAL_V,PREMIUM_TR,instrument,expiry,option_type,strike,futures_price,strike_diff
0,OPTSTK360ONE27-JAN-2026PE1240,104.90,95.50,95.50,86.15,86.15,86.15,18.75,71.0,1000.0,2.0,1149.25,1330825.0,90825.0,OPTION,JAN-2026,PE,1240,1111.0,129.0
1,OPTSTK360ONE27-JAN-2026PE1140,35.40,36.00,36.00,17.40,22.30,22.30,13.10,227.0,156000.0,312.0,1149.25,181740700.0,3900700.0,OPTION,JAN-2026,PE,1140,1111.0,29.0
2,OPTSTK360ONE24-FEB-2026PE1100,34.05,31.95,31.95,28.95,29.95,29.35,4.70,27.0,3000.0,6.0,1149.25,3389350.0,89350.0,OPTION,FEB-2026,PE,1100,1118.0,18.0
3,OPTSTK360ONE27-JAN-2026CE1200,11.10,10.55,15.00,8.80,11.45,11.45,0.35,810.0,688000.0,1376.0,1149.25,833534050.0,7934050.0,OPTION,JAN-2026,CE,1200,1111.0,89.0
4,OPTSTK360ONE27-JAN-2026CE1100,54.20,54.00,72.70,54.00,65.35,65.35,11.15,199.0,9000.0,18.0,1149.25,10462750.0,562750.0,OPTION,JAN-2026,CE,1100,1111.0,11.0


In [8]:
!pip install py_vollib


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for py_vollib: filename=py_vollib-1.0.1-py3-none-any.whl size=62838 sha256=fb9faa6acfcb85a6f5b7414ace49c79411263fea1403de8b6bafd9b008557c90
  Stored in directory: c:\users\janvi\appdata\local\pip\cache\wheels\5f\8c\ab\bbc3a2b0e6ae83633bb99f960459a2ab960b4672c7440ff4ef
  Created wheel for py_lets_be_rational: filename=py_lets_be_rational-1.0.1-py3-none-any.whl size=24460 sha256=c449e5f0d8ee4919a2f28613f71e18f17147e568b5c369d038a53b3388d2a86e
  Stored in directory: c:\users\janvi\appdata\local\pip\cache\wheels\c3\c6\50\415aabc2033e7651cc8cc590bd01c5f23f9050328f6016d6a5
Successfully built py_vollib py_lets_be_rational



[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from py_vollib.black_scholes.greeks.analytical import delta, gamma, theta, vega, rho


In [12]:
# Black–Scholes parameters
S = 3600        # Spot price (ATM)
K = 3600        # Strike price (ATM)
r = 0.065       # Risk-free rate (6.5%)
sigma = 0.20    # 20% implied volatility
T = 30 / 365    # 30 days to expiry (in years)


In [14]:
# Call option Greeks
call_greeks = {
    "Delta": delta('c', S, K, T, r, sigma),
    "Gamma": gamma('c', S, K, T, r, sigma),
    "Theta": theta('c', S, K, T, r, sigma),
    "Vega":  vega('c', S, K, T, r, sigma),
    "Rho":   rho('c', S, K, T, r, sigma),
}

# Put option Greeks
put_greeks = {
    "Delta": delta('p', S, K, T, r, sigma),
    "Gamma": gamma('p', S, K, T, r, sigma),
    "Theta": theta('p', S, K, T, r, sigma),
    "Vega":  vega('p', S, K, T, r, sigma),
    "Rho":   rho('p', S, K, T, r, sigma),
}


In [16]:
import pandas as pd

greeks_df = pd.DataFrame({
    "Call (ATM)": call_greeks,
    "Put (ATM)": put_greeks
})

greeks_df


,Call (ATM),Put (ATM)
Delta,0.548489,-0.451511
Gamma,0.001918,0.001918
Theta,-1.697567,-1.059887
Vega,4.086985,4.086985
Rho,1.547255,-1.395883


### Interpretation of Greeks

- **Delta** measures sensitivity to underlying price movement
- **Gamma** measures rate of change of Delta
- **Theta** represents time decay
- **Vega** measures sensitivity to volatility
- **Rho** measures sensitivity to interest rates

Greeks are computed using the Black–Scholes analytical model with a risk-free rate of 6.5%.


## Task 2.3: Derived Features


In [24]:
list(options_df.columns)

['CONTRACT_D',
 'PREVIOUS_S',
 'OPEN_PRICE',
 'HIGH_PRICE',
 'LOW_PRICE',
 'CLOSE_PRIC',
 'SETTLEMENT',
 'NET_CHANGE',
 'OI_NO_CON',
 'TRADED_QUA',
 'TRD_NO_CON',
 'UNDRLNG_ST',
 'NOTIONAL_V',
 'PREMIUM_TR',
 'instrument',
 'expiry',
 'option_type',
 'strike',
 'futures_price',
 'strike_diff']

In [26]:
calls_df = options_df[options_df["option_type"] == "CE"].copy()
puts_df  = options_df[options_df["option_type"] == "PE"].copy()

calls_df.shape, puts_df.shape


((6781, 20), (6109, 20))

In [28]:
calls_df[["strike", "CLOSE_PRIC", "OI_NO_CON", "TRADED_QUA"]].head()


,strike,CLOSE_PRIC,OI_NO_CON,TRADED_QUA
3,1200,11.45,810.0,688000.0
4,1100,65.35,199.0,9000.0
5,1360,1.15,112.0,23500.0
6,1300,2.00,152.0,106000.0
7,1220,7.60,315.0,207500.0


In [30]:
puts_df[["strike", "CLOSE_PRIC", "OI_NO_CON", "TRADED_QUA"]].head()


,strike,CLOSE_PRIC,OI_NO_CON,TRADED_QUA
0,1240,86.15,71.0,1000.0
1,1140,22.30,227.0,156000.0
2,1100,29.95,27.0,3000.0
8,1120,38.50,13.0,3000.0
10,1180,43.80,148.0,36500.0


## STEP 1: Total Call / Put OI and Volume

In [32]:
total_call_oi = calls_df["OI_NO_CON"].sum()
total_put_oi  = puts_df["OI_NO_CON"].sum()

total_call_vol = calls_df["TRADED_QUA"].sum()
total_put_vol  = puts_df["TRADED_QUA"].sum()

total_call_oi, total_put_oi, total_call_vol, total_put_vol


(7084626.0, 4061104.0, 10591658579.0, 5018736161.0)

## STEP 2: PCR Features

In [34]:
pcr_oi = total_put_oi / total_call_oi
pcr_volume = total_put_vol / total_call_vol

pcr_oi, pcr_volume


(0.5732277187250251, 0.4738385516835493)

## STEP 3: IV Features

In [39]:
call_iv = 0.20
put_iv = 0.20

avg_iv = (call_iv + put_iv) / 2
iv_spread = call_iv - put_iv

avg_iv, iv_spread


(0.2, 0.0)

## STEP 4: Gamma Exposure

In [45]:
mean_oi = options_df["OI_NO_CON"].mean()

gamma_exposure = S * call_greeks["Gamma"] * mean_oi
gamma_exposure


5971.696560076641

## STEP 5: Futures Basis

In [48]:
spot_close = S
futures_close = options_df["futures_price"].iloc[0]

futures_basis = (futures_close - spot_close) / spot_close
futures_basis


-0.6913888888888889

## STEP 6: Returns (Spot & Futures)

In [57]:
spot_df = pd.read_csv(r"C:\Users\Janvi\OneDrive\Desktop\Quant_Trading_Assignment\data\nifty spot 5min.csv")
spot_df["timestamp"] = pd.to_datetime(spot_df["timestamp"])
spot_df = spot_df.sort_values("timestamp")


In [59]:
spot_return = spot_df["close"].pct_change().iloc[-1]
futures_return = options_df["futures_price"].pct_change().iloc[-1]

spot_return, futures_return


C:\Users\Janvi\AppData\Local\Temp\ipykernel_24336\1923282455.py:2: FutureWarning: The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  futures_return = options_df["futures_price"].pct_change().iloc[-1]


(-0.0041754453169892125, 0.0)

## STEP7: Delta Neutral Ratio

In [61]:
delta_neutral_ratio = abs(call_greeks["Delta"]) / abs(put_greeks["Delta"])
delta_neutral_ratio


1.2147835178397994

## TASK 2.4

In [68]:
final_df = spot_df.copy()


In [72]:
list(spot_df.columns)


['timestamp', 'open', 'high', 'low', 'close', 'volume']

In [74]:
final_df["ema_5"] = final_df["close"].ewm(span=5, adjust=False).mean()
final_df["ema_15"] = final_df["close"].ewm(span=15, adjust=False).mean()


In [76]:
spot_return


-0.0041754453169892125

In [78]:
final_df["spot_return"] = final_df["close"].pct_change()


In [80]:
final_df["futures_return"] = futures_return


In [90]:
final_df["delta_neutral_ratio"] = delta_neutral_ratio
final_df["gamma_exposure"] = gamma_exposure
final_df["futures_basis"] = futures_basis

# IV features
final_df["avg_iv"] = avg_iv
final_df["iv_spread"] = iv_spread

# PCR features
final_df["pcr_oi"] = pcr_oi
final_df["pcr_volume"] = pcr_volume


In [92]:
final_df.columns


Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'ema_5',
       'ema_15', 'spot_return', 'futures_return', 'delta_neutral_ratio',
       'gamma_exposure', 'futures_basis', 'avg_iv', 'iv_spread', 'pcr_oi',
       'pcr_volume'],
      dtype='object')

In [94]:
final_df = final_df.dropna()

final_df.to_csv("nifty_features_5min.csv", index=False)


In [96]:
final_df.columns


Index(['timestamp', 'open', 'high', 'low', 'close', 'volume', 'ema_5',
       'ema_15', 'spot_return', 'futures_return', 'delta_neutral_ratio',
       'gamma_exposure', 'futures_basis', 'avg_iv', 'iv_spread', 'pcr_oi',
       'pcr_volume'],
      dtype='object')

In [98]:
final_df.to_csv(
    "../data/clean/nifty_features_5min.csv",
    index=False
)
